# pytorch environment

In [1]:
import os
import re
import pandas as pd
import datefinder #pip install datefinder if not found

# 0. Load data

In [353]:
def clean(raw_text, stop_words = []):
    ### 
    
    # clean raw_texts by removing special character and digits.
    # remove stop words if specified.
    # and lemmatize followed by stem operation for each word.
    
    ###
            
    text = raw_text.replace('\xa0', ' ').replace('\n', ' ').lower()

    # remove special characters such as the "'" in "it's".
    text = re.sub(r'x\.\.\.', 'xxx', text)

    text = re.sub(r'y\.\.\.', 'yyy', text)

    text = re.sub(r'\W', ' ', text)

    # remove single character such as the "s" in "it s".
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # remove stop words if it's not none.
    for w in stop_words:
        text = re.sub(r"\b" + w + r"\b", '', text)

    # unify successive blank space as one blank space.
    text = re.sub(r'\s+', ' ', text, flags=re.I)
        

    return text

In [354]:
#x_train data
x_tr_df = pd.read_csv('train_folder/x_train_ids.csv', index_col=0)

In [355]:
# author code
tr_dir = "train_folder/txt_files/"
tr_files = x_tr_df['filename'].values


tr_texts = []
for tr_file in tr_files:
    f = open(os.path.join(tr_dir, tr_file), "r")
    ff = f.read()
    ff = clean(ff)
    tr_texts.append(ff)
# tr_texts has all the texts in it

In [269]:
#y_train data
y_tr_df = pd.read_csv('./Y_train_predilex.csv', index_col=0)

In [270]:
y_tr_df.head()

,sexe,date_accident,date_consolidation
ID,,,
0,homme,1991-04-09,n.c.
1,homme,2005-06-10,2010-01-19
2,femme,1997-09-26,n.c.
3,femme,1982-08-07,1982-11-07
4,homme,1996-11-26,n.c.


# 1. Find sentence with date in it

In [356]:
all_dates = []
for i in range(len(tr_texts)):
    if y_tr_df['date_accident'][i] != 'n.c.':
        ff = [m.start() for m in re.finditer(y_tr_df['date_accident'][i][:4], tr_texts[i])]
    else:
        ff = []
    all_dates.append(ff)

In [366]:
sentences = []
for ind in range(len(tr_texts)):
    sentence = []
    window = 20
    for date in all_dates[ind]:
        left = tr_texts[ind][date - window : date]
        right = tr_texts[ind][date + 4 : date + window]
        left_num = re.findall(r'\d', left)
        right_num = re.findall(r'\d', right)
        if len(left_num) == 0 & len(right_num) == 0:
            pass
        else:
            text = tr_texts[ind][date - window : date + window]
            sentence.append(text)
    sentences.append(sentence)

In [372]:
pattern1 = r"\d+ \w+ \d+"
pattern2 = r"1° \w+ \d+"
pattern3 = r"1er \w+ \d+"
pattern4 = r"\d+\.\d+\.\d+"
pattern5 = r"\d+/\d+/\d+"
pattern6 = r"\d+ / \d+ / \d+"
pattern7 = r"\d+/ \d+/ \d+"
pattern8 = r"\d+\. \d+\. \d+"

patterns = [pattern1, pattern3] #, pattern3, pattern4, pattern5, pattern6, pattern7, pattern8] 

In [373]:
dates = []
fails = []
for i in range(len(sentences)):
    date = []
    index = []
    for sentence in sentences[i]:
        l = []
        for pattern in patterns:
            l.extend(re.findall(pattern, sentence))
        if len(l) == 0:
            fails.append(sentence)
        date.extend(l)
    dates.append(date)

In [374]:
len(fails)

69

In [375]:
fails

['re 1997 à septembre 1998 qu ainsi son pr',
 '2 au titre de année 2002 de sorte que le',
 ' survenu le 1o mars 2002 est pas objet d',
 'res en 1996 puis en 1997 mme myriam yyy ',
 'let 1996 et janvier 1997 aurait pas refu',
 'let 1996 et janvier 1997 de sorte qu il ',
 'tions de 1996 et de 1997 ni la qualité d',
 'ables de 1996 et de 1997 date de consoli',
 '0 outre la somme de 2000 au titre de art',
 'il 2008 à septembre 2008 secondairement ',
 'obre 2007 à février 2008 ainsi que la so',
 'mbre 2008 à février 2009 du fait de troi',
 'conclu le 15décembre1992 que accident po',
 's monsieur gilbert 72000 le mans la mutu',
 ' 692 euros en avril 2004 et de 1 566 eur',
 ' 1 566 euros en mai 2004 il produit deux',
 'rier 1984 à ajaccio 20000 20160 coggia a',
 ' finosello cs 15002 20000 ajaccio cedex ',
 'r pour 235 jours en 2002 8 ans et 6 mois',
 'n mars 2008 et juin 2010 et il ne peut d',
 'août 1984 à ajaccio 20090 ajaccio ayant ',
 '52 euros pour année 2006 en conséquence ',
 '999 alor

# Test pattern on test data 

In [376]:
#x_test data
x_te_df = pd.read_csv('test_folder/x_test_ids.csv', index_col=0)
te_dir = "test_folder/txt_files/"
te_files = x_te_df['filename'].values
te_texts = []
for te_file in te_files:
    f = open(os.path.join(te_dir, te_file), "r")
    ff = f.read()
    ff = clean(ff)
    te_texts.append(ff)

In [379]:
patterns

['\\d+ \\w+ \\d+', '1er \\w+ \\d+']

In [377]:
index = []
for pattern in patterns:
    index.extend(re.finditer(pattern, tr_texts[0]))

In [378]:
window = 50
for ind in index:
    chunk = tr_texts[0][ind.start() - window: ind.end() + window]
    print('--------------')
    print(chunk)

--------------

--------------
r appel agen chambre sociale audience publique du 30 avril 2002 de rg 01 00515 republique francaise au nom du peu
--------------
ique francaise au nom du peuple francais arret du 30 avril 2002 01 00515 yvon xxx annie xxx groupe azur assurance
--------------
ociale dans affaire entre monsieur yvon xxx né le 30 septembre 1955 à tonneins 47 yyy du laurier 47300 bias rep assis
--------------
 des affaires de sécurité sociale agen en date du 12 février 2001 une part et groupe azur assurances iard 7 avenue 
--------------
e été débattue et plaidée en audience publique le 19 mars 2002 devant monsieur milhet président de chambre madam
--------------
isées de la date à laquelle arrêt serait rendu le 9 avril 1991 monsieur xxx été victime un accident du travail a
--------------
 été gravement blessé suivant jugement en date du 24 avril 1997 confirmé par arrêt de la cour en date du 30 mars 
--------------
ril 1997 confirmé par arrêt de la cour en date du 30 mars 1999

In [319]:
y_tr_df['date_accident'][0]

'1991-04-09'

## 1.2 Using Datefinder Module

In [165]:
# doesnt work because Septembre is in french and wont be found by regular pattern such as 9 September

matches = datefinder.find_dates(tr_texts[0], index = True)

all_dates = []
for match in matches:
    if match[0].year > 1900 and match[0].year < 2100:
        print(match)
        all_dates.append(match[1])
#all_dates

all_dates

ind = 9
tr_texts[5][all_dates[ind][0]:all_dates[ind][1]]

for i in range(len(tr_texts[0])):
    if tr_texts[0][i] == '.':
        print(i)

all_dates